In [ ]:
# 문제: 심장병 데이터셋에서 Z-score가 3보다 큰 이상치가 가장 많이 나타나는 컬럼과 그 컬럼의 이상치 개수 구하기

import pandas as pd
import numpy as np

df = pd.read_csv("/content/drive/MyDrive/datasets/heart_disease_data.csv")

# 결과 컬럼과 이상치 개수가 저장될 변수
result_col = ''
result_out = 0

# 컬럼별로 Z-score가 3보다 큰 이상치를 구해서 가장 개수가 많은 컬럼 저장
for i in df.columns:
  col_mean = df[i].mean()
  col_std = df[i].std()

  # Z-score 연산
  z_score = np.abs( (df[i] - col_mean) / col_std )

  # Z-score가 3보다 큰 이상치의 개수 저장
  outlier = df[z_score > 3]

  # 이상치가 가장 많은 컬럼과 이상치 개수 저장
  if len(outlier) > result_out:
    result_col = i
    result_out = len(outlier)

# 결과 출력
print(result_col)
print(result_out)

ca
18


In [ ]:
# 문제: 심장병 데이터셋에서 'chol' 컬럼의 Z-score를 구한 후 3보다 큰 이상치의 개수와 그 이상치 중 가장 큰 이상치의 값 구하기
# (가장 큰 이상치의 값은 절대값을 기준으로 구하며, 반올림하여 소수점 3자리까지 출력한다.)

import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/datasets/heart_disease_data.csv")

from scipy.stats import zscore

# Z-score로 만든 후 새로운 컬럼에 넣기
df['chol_z'] = zscore(df['chol'])

# Z-score가 3보다 큰 이상치 개수 저장
outlier_cnt = df['chol_z'][abs(df['chol_z']) > 3].count()

# Z-score가 3보다 큰 이상치 중 가장 큰 값 저장
outlier_value = df['chol_z'][abs(df['chol_z']) > 3].sort_values(ascending=False).iloc[0]

# 결과 출력
print(outlier_cnt)
print(round(outlier_value, 3))

13
6.167


In [ ]:
# 문제: 심장병 데이터셋에서 IQR 연산으로 ca 컬럼의 이상치 개수 구하기

import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/datasets/heart_disease_data.csv")

# 1사분위수, 3사분위수 저장
q1 = df['ca'].quantile(0.25)
q3 = df['ca'].quantile(0.75)

#  IQR, 1사분위수 하한값, 3사분위수 상한값 저장
iqr = q3 - q1
low = q1 - 1.5 * iqr
upp = q3 + 1.5 * iqr

# 이상치만 저장 후 개수 출력
outlier = df['ca'][ (df['ca'] < low) | (df['ca'] > upp) ]
print(len(outlier))

87


In [ ]:
# 문제: 각 구급 보고서별 출동시각과 신고시각의 차이를 '소요시간' 칼럼으로 만들어 초(sec) 단위로 구하고
# 소방서명별 소요시간의 평균을 오름차순으로 정렬했을 때, 3번째로 작은 소요시간의 값과 소방서명을 출력하시오.

import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/datasets/report_data.csv")

# 날짜형으로 만든 '신고일시' 컬럼 생성
df['신고일시'] = pd.to_datetime(df['신고일자'].astype(str) + ' ' + df['신고시각'].astype(str).str.zfill(6), format='%Y%m%d %H%M%S')

# 날짜형으로 만든 '출동일시' 컬럼 생성
df['출동일시'] = pd.to_datetime(df['출동일자'].astype(str) + ' ' + df['출동시각'].astype(str).str.zfill(6), format='%Y%m%d %H%M%S')

# 신고 및 출동 간격을 초 단위로 구한 '소요시간' 컬럼 생성
df['소요시간'] = (df['출동일시'] - df['신고일시']).dt.total_seconds()

# 소방서명별 소요시간 평균 계산 후 오름차순 정렬한 데이터프레임 생성
fire_station_time = df.groupby('소방서명')['소요시간'].mean().reset_index().sort_values(by='소요시간')

# 3번째로 작은 소요시간과 소방서명 출력
print(fire_station_time.iloc[2]['소방서명'])
print(fire_station_time.iloc[2]['소요시간'])

종로소방서
175.5


In [ ]:
# 문제: airquality 데이터를 활용하여 결측값이 가장 많은 컬럼명을 확인하여 결측값을 최빈값으로 대체한 후
# 결측값을 대체하기 전의 컬럼 평균과 후의 컬럼 평균의 차이 구하기 (반올림하여 소수점 3자리까지 출력한다.)

import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/datasets/airquality.csv")

# 결측값이 가장 많은 컬럼명 저장
miss = df.isnull().sum().sort_values(ascending=False).index[0]

# 결측값 대체 전 컬럼 평균 저장
before = df[miss].mean()

# 결측값 최빈값으로 대체하기
df[miss].fillna(df[miss].mode().iloc[0], inplace=True)

# 결측값 대체 후 컬럼 평균 저장
after = df[miss].mean()

# 차이를 구한 후 반올림하여 소수점 3자리까지 출력
result = round(abs(before-after), 3)
print(result)

4.626


In [2]:
# 문제: 와인 데이터셋에서 alcohol 컬럼을 최소-최대 정규화하고, alcohol 컬럼의 표준편차 구하기
# 자유도는 0, 연산 결과는 반올림 후 소수점 3자리까지 출력

import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/datasets/wine_data.csv")

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# alcohol 컬럼을 최소-최대 정규화
df['alcohol'] = scaler.fit_transform(df[['alcohol']])

# 자유도 0의 표준편차를 구하고 소수점 3자리까지 반올림하여 출력
result = df['alcohol'].std(ddof=0)
print(round(result, 3))

0.213


In [3]:
# 문제: COVID-19 관련 데이터에서 Deaths 컬럼을 라벨인코딩하여, 가장 큰 사망자 라벨값과 5번째로 적은 사망자 라벨값의 차이 구하기

import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/datasets/covid_death_bycountry.csv")

from sklearn.preprocessing import LabelEncoder

# Deaths 컬럼 라벨인코딩
le = LabelEncoder()
df['Deaths'] = le.fit_transform(df['Deaths'])

# Death 컬럼을 기준으로 내림차순 정렬
df.sort_values('Deaths', ascending=False, inplace=True)

# 가장 큰 사망자 라벨값과 5번째로 적은 사망자 라벨값 저장
dead_max = df['Deaths'].iloc[0]
dead_fiv = df['Deaths'].iloc[-5]

# 결과 출력
print(dead_max - dead_fiv)

204


In [21]:
# 문제: 2000~2022년 국내 데이터셋에서 2008년 7월 1일 출생률이 가장 높은 지역의 이름 출력
# 결측치는 중앙값으로 대체한다.

import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/datasets/Korean_demographics_2000-2022.csv")

# 2008년 7월 1일 데이터만 담은 데이터프레임 생성
df2 = df[:][df['Date'] == '7/1/2008']

# 결측치가 있는 컬럼들의 이름 저장
miss = df2.columns[df2.isna().any()]

# 각 컬럼의 결측치를 중앙값으로 대체하는 반복문
for i in miss:
  df2[i] = df2[i].fillna(df2[i].median())

# 출생률이 높은 순서로 내림차순 정렬
df2 = df2.sort_values('Birth_rate', ascending=False)

# 출생률이 가장 높은 지역명 출력
result = df2['Region'].iloc[0]
print(result)

Gyeonggi-do


In [38]:
# 문제: 수면 데이터셋에서 Sleep Disorder 컬럼과 가장 높은 상관계수를 갖는 변수의 최빈값 출력

import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/datasets/Sleep_health_and_lifestyle_dataset.csv")

from sklearn.preprocessing import LabelEncoder

# 정수형 컬럼 라벨 인코딩
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])
df['Occupation'] = le.fit_transform(df['Occupation'])
df['BMI Category'] = le.fit_transform(df['BMI Category'])
df['Blood Pressure'] = le.fit_transform(df['Blood Pressure'])
df['Sleep Disorder'] = le.fit_transform(df['Sleep Disorder'])

# 데이터의 상관계수 저장
data_corr = df.corr()

# Sleep Disorder 컬럼과 다른 컬럼 간의 상관계수를 절대값으로 내림차순 저장
data_corr_sort = abs(data_corr['Sleep Disorder']).sort_values(ascending=False)

# Sleep Disorder 컬럼과 가장 상관계수가 높은 다른 컬럼명 저장
# index[0]은 자기 자신 컬럼이므로 index[1]
max_corr = data_corr_sort.index[1]

# Sleep Disorder 컬럼과 상관계수가 가장 높은 컬럼의 최빈값 출력
result = df[max_corr].mode().iloc[0]
print(result)

0
